In [13]:
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from pinecone import (
    Pinecone
)

class EmbeddingMapper:
    def __init__(self, model_name: str, pinecone_api_key: str, index_name: str, metadata_key: str = "ticker"):
        # Initialize the embedding model
        self.model = SentenceTransformer(model_name)
        
        pc = Pinecone(api_key=pinecone_api_key)
        self.index = pc.Index(index_name)
        self.metadata_key = metadata_key
        
    def get(self, query):
        query_embedding = self.model.encode(query, convert_to_numpy=True)
        
        # Search in Pinecone
        results = self.index.query(
            vector=query_embedding.tolist(),
            top_k=1,
            include_metadata=True
        )
        
        if results.matches:
            return results.matches[0].metadata[self.metadata_key]
        return None

# Initialize mappers for all three types
load_dotenv()
pc_api_key = os.getenv("PINECONE_API_KEY")

# Stock mapper
stock_mapper = EmbeddingMapper(
    model_name="all-MiniLM-L6-v2",
    pinecone_api_key=pc_api_key,
    index_name="stock-index",
    metadata_key="ticker"
)

# Crypto mapper
crypto_mapper = EmbeddingMapper(
    model_name="all-MiniLM-L6-v2",
    pinecone_api_key=pc_api_key,
    index_name="crypto-index",
    metadata_key="ticker"
)

# City mapper
city_mapper = EmbeddingMapper(
    model_name="all-MiniLM-L6-v2",
    pinecone_api_key=pc_api_key,
    index_name="city-index",
    metadata_key="city"
)

In [14]:
# Test Stock Mapper
test_queries = ["AAPL", "Apple Inc.", "apple", "What is the current stock price of Tesla.", "Google", "google", "TSLA", "Tesla", "tesla", "Microsoft Corporation", "microsoft"]

print("📊 Stock Mapper Tests:")
for query in test_queries:
    ticker = stock_mapper.get(query)
    print(f"Query: {query} -> Ticker: {ticker}")

📊 Stock Mapper Tests:
Query: AAPL -> Ticker: AAPL
Query: Apple Inc. -> Ticker: AAPL
Query: apple -> Ticker: AAPL
Query: What is the current stock price of Tesla. -> Ticker: TSLA
Query: Google -> Ticker: GOOGL
Query: google -> Ticker: GOOGL
Query: TSLA -> Ticker: TSLA
Query: Tesla -> Ticker: TSLA
Query: tesla -> Ticker: TSLA
Query: Microsoft Corporation -> Ticker: MSFT
Query: microsoft -> Ticker: MSFT


In [16]:
# Test Crypto Mapper
test_queries = ["BTC", "bitcoin", "Do you know the price of cardano?", "What is current cost of ETH", "solana", "SOL", "Do you know the price of solana?"]

print("💰 Crypto Mapper Tests:")
for query in test_queries:
    ticker = crypto_mapper.get(query)
    print(f"Query: {query} -> Ticker: {ticker}")

💰 Crypto Mapper Tests:
Query: BTC -> Ticker: BTC
Query: bitcoin -> Ticker: BTC
Query: Do you know the price of cardano? -> Ticker: ADA
Query: What is current cost of ETH -> Ticker: ETH
Query: solana -> Ticker: SOL
Query: SOL -> Ticker: SOL
Query: Do you know the price of solana? -> Ticker: SOLKIT


In [17]:
# Test your specific query
test_query = "What is the price of bitcoin currently?"
ticker = crypto_mapper.get(test_query)
print(f"Query: '{test_query}'")
print(f"Extracted Ticker: {ticker}")
print(f"\n✅ RAG Mapper successfully extracted: {ticker} from the query!")


Query: 'What is the price of bitcoin currently?'
Extracted Ticker: BTC

✅ RAG Mapper successfully extracted: BTC from the query!


In [18]:
# Complete workflow: RAG + Fine-tuned Model Integration
from transformers import pipeline

# Step 1: Initialize the fine-tuned model
model_id = "Mozes721/crypto-stock-weather-agent"
generator = pipeline(
    "text2text-generation",
    model=model_id,
    max_length=256,
    num_return_sequences=1,
    do_sample=False
)

# Step 2: User query
user_query = "What is the price of bitcoin currently?"

# Step 3: RAG extraction - Extract ticker from query
extracted_ticker = crypto_mapper.get(user_query)
print(f"🔍 Step 1 - RAG Extraction:")
print(f"   Query: '{user_query}'")
print(f"   Extracted: {extracted_ticker}\n")

# Step 4: Simulate fetching actual data (in real app, you'd call an API)
# For demo, let's use a mock price
mock_price = 45000  # In real app: price = fetch_crypto_price(extracted_ticker)

# Step 5: Format for fine-tuned model (as it was trained)
formatted_prompt = f"{user_query} | Values: price={mock_price}"
print(f"📝 Step 2 - Format for Fine-tuned Model:")
print(f"   Formatted: '{formatted_prompt}'\n")

# Step 6: Generate natural language response using fine-tuned model
response = generator(formatted_prompt)
generated_text = response[0]['generated_text']
print(f"🤖 Step 3 - Fine-tuned Model Response:")
print(f"   {generated_text}\n")

print("=" * 60)
print("✅ Complete workflow: RAG extracts → Fetch data → Fine-tuned model generates response")


Device set to use mps:0


🔍 Step 1 - RAG Extraction:
   Query: 'What is the price of bitcoin currently?'
   Extracted: BTC

📝 Step 2 - Format for Fine-tuned Model:
   Formatted: 'What is the price of bitcoin currently? | Values: price=45000'

🤖 Step 3 - Fine-tuned Model Response:
   Bitcoin is a cryptocurrency that has a price range of $45,000,000 and a price range of $45,000,000.

✅ Complete workflow: RAG extracts → Fetch data → Fine-tuned model generates response


In [19]:
# Test City Mapper
test_queries = ["What is current weather in New York?", "What is it like in Warsaw right now?", "What is the weather in Tokyo?", "What is the weather in London?", "I want to know how is Riga doing right now weather wise?"]

print("🌍 City Mapper Tests:")
for query in test_queries:
    city = city_mapper.get(query)
    print(f"Query: {query} -> City: {city}")

🌍 City Mapper Tests:
Query: What is current weather in New York? -> City: New York
Query: What is it like in Warsaw right now? -> City: Warsaw
Query: What is the weather in Tokyo? -> City: Tokyo
Query: What is the weather in London? -> City: London
Query: I want to know how is Riga doing right now weather wise? -> City: Rīga
